In [1]:
import pandas as pd
import numpy as np
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.utils import np_utils
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from numpy import expand_dims
from tensorflow.keras.applications.vgg16 import preprocess_input

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from tensorflow.keras.applications.inception_v3 import InceptionV3
from glob import glob

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import os.path
from tensorflow.keras.models import load_model

import operator
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pylab as pl

import seaborn as sns
import matplotlib.pyplot as plt   
from IPython.display import Image, display
import matplotlib.cm as cm

import warnings
warnings.filterwarnings("ignore")

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
print('Number of GPUs: ', len(tf.config.list_physical_devices('GPU')))
tf.config.experimental.set_memory_growth(physical_devices[0],True)

Number of GPUs:  1


# Model Predictions Dataframe Initialisation


In [3]:
def open_images(paths):
    '''
    Opens a batch of images, given the image path(s) as a list
    '''
    images = []
    for path in paths:
        image = load_img(path, target_size=(224,224))
        image = np.array(image)/255.0
        images.append(image)
    return np.array(images)

def get_labels(paths):
    '''
    it is possible to get the label from the path, just split the path by "/" and index -2
    For example, /kaggle/input/brain-tumor-mri-dataset/Training/pituitary/Tr-pi_1020.jpg
    splitting by "/" gives ['kaggle','input','brain-tumor-mri-dataset','Training','pituitary','Tr-pi_1020.jpg']
    Now indexing -2 gives "pituitary"
    '''
    label = []
    for path in paths:
        path = path.split('/')[-2]
        label.append(labels.index(path))
    return label

train_dir = r'Training/'
test_dir = r'Testing/'

train_paths = []
test_paths = []


for label in os.listdir(train_dir):
    for file in os.listdir(train_dir+label):
        train_paths.append(train_dir+label+'/'+file)


for label in os.listdir(test_dir):
    for file in os.listdir(test_dir+label):
        test_paths.append(test_dir+label+'/'+file)

labels_test = os.listdir(test_dir)
labels_train = os.listdir(train_dir)

model_predictions_train = pd.DataFrame(columns = ['3c2f','vgg16','vgg16_ctrained','resnet','resnet_ctrained','inception','inception_ctrained','actual'], 
                                       index = train_paths ).fillna('here')

model_predictions_test = pd.DataFrame(columns = ['3c2f','vgg16','vgg16_ctrained','resnet','resnet_ctrained','inception','inception_ctrained','actual'], 
                                       index = test_paths ).fillna('here')

## Loading Data and Augmenting Data

In [4]:
generator_train = ImageDataGenerator(rescale=1/255,
                                     rotation_range=7,
                                     horizontal_flip=True,
                                     shear_range=0.1,
                                     height_shift_range=0.07,
                                     zoom_range=0.1)


train = generator_train.flow_from_directory(r'Training', target_size=(224,224), # height and width of images to feed into CNN
                                              batch_size=16, class_mode= "categorical", color_mode='rgb')


generator_test = ImageDataGenerator(rescale=1/255,
                                     rotation_range=7,
                                     horizontal_flip=True,
                                     shear_range=0.1,
                                     height_shift_range=0.07,
                                     zoom_range=0.1)


test = generator_test.flow_from_directory(r'Testing', target_size=(224,224),
                                              batch_size=16, class_mode= "categorical", color_mode='rgb')

IMAGE_SIZE = [224, 224]

train_path = r'Training'
valid_path = r'Testing'

folders = glob(r'Training/*')

Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


# Model Predictions Dataframe Initialisation

In [ ]:
model1 = Sequential()

# Convolutional layer 1
model1.add(Conv2D(filters = 32, kernel_size = (3,3), input_shape=(224, 224, 3), activation='relu'))
model1.add(BatchNormalization())
model1.add(MaxPooling2D(pool_size=(2,2), strides = 2))

# Convolutional layer 2
model1.add(Conv2D(64,(3,3), activation='relu'))
model1.add(BatchNormalization())
model1.add(MaxPooling2D(pool_size=(2,2)))

# Convolutional layer 3
model1.add(Conv2D(128,(3,3), activation='relu'))
model1.add(BatchNormalization())
model1.add(MaxPooling2D(pool_size=(2,2)))

model1.add(Flatten())

# Feedforward
model1.add(Dense(units= 512, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(units=512, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(units=4, activation='softmax'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, decay=0.0001, clipvalue=0.5)
model1.compile(optimizer=optimizer, loss='categorical_crossentropy',
                   metrics= ['categorical_accuracy'])

model1_es = EarlyStopping(monitor = 'loss', min_delta = 1e-11, patience = 12, verbose = 1)
model1_rlr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 6, verbose = 1)
model1_mcp = ModelCheckpoint(filepath = r'temp\model1_weights.h5', monitor = 'categorical_accuracy',
                             save_best_only = True, verbose = 1)

history1 = model1.fit(train, steps_per_epoch=5712//16, epochs=120, validation_data=test, validation_steps= 1311//16, callbacks=[model1_es, model1_rlr, model1_mcp])


In [7]:
if os.path.isfile('models/self_3conv_2ff.h5') is False:
    model1.save('models/self_3conv_2ff.h5')
    
self_3conv_2ff = load_model('models/self_3conv_2ff.h5')

# Model 2 - VGG16 (No Convolution Trained)


In [8]:
vgg16_conv = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in vgg16_conv.layers:
    layer.trainable = False

vgg_feedforward = Flatten()(vgg16_conv.output)
vgg_feedforward = Dense(4096,activation = 'relu')(vgg_feedforward)
vgg_feedforward = Dense(1024,activation = 'relu')(vgg_feedforward)

prediction = Dense(4, activation='softmax')(vgg_feedforward)

model_vgg16 = Model(inputs=vgg16_conv.input, outputs=prediction)

model_vgg16.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['categorical_accuracy']
)


model1_es = EarlyStopping(monitor = 'loss', min_delta = 1e-11, patience = 12, verbose = 1)
model1_rlr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 6, verbose = 1)

# Automatically saves the best weights of the model, based on best val_accuracy
model1_mcp = ModelCheckpoint(filepath = r'temp\model1_weights.h5', monitor = 'categorical_accuracy',
                             save_best_only = True, verbose = 1)

vgg16_fit = model_vgg16.fit(
  train,
  validation_data=test,
  epochs=100,
  steps_per_epoch=5712//32,
  validation_steps=1311//32,
  callbacks=[model1_es, model1_rlr, model1_mcp])

58889256/58889256 [==============================] - 3s 0us/step
Epoch 1/100
178/178 [==============================] - ETA: 0s - loss: 1.5659 - categorical_accuracy: 0.7303
Epoch 1: categorical_accuracy improved from -inf to 0.73034, saving model to temp\model1_weights.h5
178/178 [==============================] - 64s 352ms/step - loss: 1.5659 - categorical_accuracy: 0.7303 - val_loss: 0.5245 - val_categorical_accuracy: 0.8047 - lr: 0.0010
Epoch 2/100
178/178 [==============================] - ETA: 0s - loss: 0.3875 - categorical_accuracy: 0.8536
Epoch 2: categorical_accuracy improved from 0.73034 to 0.85358, saving model to temp\model1_weights.h5
178/178 [==============================] - 69s 389ms/step - loss: 0.3875 - categorical_accuracy: 0.8536 - val_loss: 0.6314 - val_categorical_accuracy: 0.7531 - lr: 0.0010
Epoch 3/100
178/178 [==============================] - ETA: 0s - loss: 0.3343 - categorical_accuracy: 0.8764
Epoch 3: categorical_accuracy improved from 0.85358 to 0.87640,

In [9]:
if os.path.isfile('models/vgg16_4096_1024.h5') is False:
    model1.save('models/vgg16_4096_1024.h5')

vgg16_4096_1024 = load_model('models/vgg16_4096_1024.h5')
print(vgg16_4096_1024.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 batch_normalization_3 (Batc  (None, 222, 222, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 batch_normalization_4 (Batc  (None, 109, 109, 64)     256       
 hNormalization)                                                 
                                                      

# Model 3 - VGG16 (Last Convolution Trained)

In [10]:
vgg16_conv = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in vgg16_conv.layers[:15]:
    layer.trainable = False

vgg_feedforward = Flatten()(vgg16_conv.output)
vgg_feedforward = Dense(64,activation = 'relu')(vgg_feedforward)
vgg_feedforward = Dense(16,activation = 'relu')(vgg_feedforward)

prediction = Dense(len(folders), activation='softmax')(vgg_feedforward)

model_vgg16 = Model(inputs=vgg16_conv.input, outputs=prediction)

model_vgg16.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['categorical_accuracy']
)


model1_es = EarlyStopping(monitor = 'loss', min_delta = 1e-11, patience = 12, verbose = 1)
model1_rlr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 6, verbose = 1)

# Automatically saves the best weights of the model, based on best val_accuracy
model1_mcp = ModelCheckpoint(filepath = r'temp\model1_weights.h5', monitor = 'categorical_accuracy',
                             save_best_only = True, verbose = 1)

vgg16_fit = model_vgg16.fit(
  train,
  validation_data=test,
  epochs=100,
  steps_per_epoch=5712//32,
  validation_steps=1311//32,
  callbacks=[model1_es, model1_rlr, model1_mcp])

Epoch 1/100
178/178 [==============================] - ETA: 0s - loss: 1.4071 - categorical_accuracy: 0.2637
Epoch 1: categorical_accuracy improved from -inf to 0.26369, saving model to temp\model1_weights.h5
178/178 [==============================] - 62s 339ms/step - loss: 1.4071 - categorical_accuracy: 0.2637 - val_loss: 1.3822 - val_categorical_accuracy: 0.3281 - lr: 0.0010
Epoch 2/100
178/178 [==============================] - ETA: 0s - loss: 1.3850 - categorical_accuracy: 0.2707
Epoch 2: categorical_accuracy improved from 0.26369 to 0.27072, saving model to temp\model1_weights.h5
178/178 [==============================] - 61s 340ms/step - loss: 1.3850 - categorical_accuracy: 0.2707 - val_loss: 1.3825 - val_categorical_accuracy: 0.3203 - lr: 0.0010
Epoch 3/100
178/178 [==============================] - ETA: 0s - loss: 1.3831 - categorical_accuracy: 0.2820
Epoch 3: categorical_accuracy improved from 0.27072 to 0.28195, saving model to temp\model1_weights.h5
178/178 [================

In [11]:
if os.path.isfile('models/vgg16_64_16_last_conv_train.h5') is False:
    model_vgg16.save('models/vgg16_64_16_last_conv_train.h5')

vgg16_64_16_last_conv_trained = load_model('models/vgg16_64_16_last_conv_train.h5')
print(vgg16_64_16_last_conv_trained.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

# Model 4 - ResNet150 (No Convolution Trained) 

In [4]:
ResNet_conv = ResNet152V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in ResNet_conv.layers:
    layer.trainable = False

resnet_feedforword = Flatten()(ResNet_conv.output)

prediction = Dense(len(folders), activation='softmax')(resnet_feedforword)

model_ResNet = Model(inputs= ResNet_conv.input, outputs=prediction)

model_ResNet.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['categorical_accuracy']
)

model1_es = EarlyStopping(monitor = 'loss', min_delta = 1e-11, patience = 12, verbose = 1)
model1_rlr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 6, verbose = 1)
model1_mcp = ModelCheckpoint(filepath = r'temp\model1_weights.h5', monitor = 'categorical_accuracy',
                             save_best_only = True, verbose = 1)

model_ResNet_fit = model_ResNet.fit(
  train,
  validation_data=test,
  epochs=100,
  steps_per_epoch=5712//32,
  validation_steps=1311//32,
  callbacks=[model1_es, model1_rlr, model1_mcp])

234545216/234545216 [==============================] - 24s 0us/step
Epoch 1/100
178/178 [==============================] - ETA: 0s - loss: 3.8381 - categorical_accuracy: 0.7900
Epoch 1: categorical_accuracy improved from -inf to 0.79003, saving model to temp\model1_weights.h5
178/178 [==============================] - 107s 532ms/step - loss: 3.8381 - categorical_accuracy: 0.7900 - val_loss: 4.1131 - val_categorical_accuracy: 0.8141 - lr: 0.0010
Epoch 2/100
178/178 [==============================] - ETA: 0s - loss: 3.0886 - categorical_accuracy: 0.8652
Epoch 2: categorical_accuracy improved from 0.79003 to 0.86517, saving model to temp\model1_weights.h5
178/178 [==============================] - 69s 386ms/step - loss: 3.0886 - categorical_accuracy: 0.8652 - val_loss: 3.2448 - val_categorical_accuracy: 0.8703 - lr: 0.0010
Epoch 3/100
178/178 [==============================] - ETA: 0s - loss: 2.7640 - categorical_accuracy: 0.8908
Epoch 3: categorical_accuracy improved from 0.86517 to 0.89

In [ ]:
if os.path.isfile('models/resnet152.h5') is False:
    model_ResNet.save('models/resnet152.h5')

resnet152 = load_model('models/resnet152.h5')
print(resnet152.summary())

# Model 5 - ResNet150 (Last 2 Convolution Trained)


In [5]:
ResNet_conv = ResNet152V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in ResNet_conv.layers[-7:]:
    layer.trainable = False    

resnet_feedforword = Flatten()(ResNet_conv.output)

prediction = Dense(len(folders), activation='softmax')(resnet_feedforword)

model_ResNet = Model(inputs= ResNet_conv.input, outputs=prediction)

model_ResNet.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['categorical_accuracy']
)

model1_es = EarlyStopping(monitor = 'loss', min_delta = 1e-11, patience = 12, verbose = 1)
model1_rlr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 6, verbose = 1)
model1_mcp = ModelCheckpoint(filepath = r'temp\model1_weights.h5', monitor = 'categorical_accuracy',
                             save_best_only = True, verbose = 1)

model_ResNet_fit = model_ResNet.fit(
  train,
  validation_data=test,
  epochs=100,
  steps_per_epoch=5712//32,
  validation_steps=1311//32,
  callbacks=[model1_es, model1_rlr, model1_mcp])

Epoch 1/100
178/178 [==============================] - ETA: 0s - loss: 2.2147 - categorical_accuracy: 0.5358
Epoch 1: categorical_accuracy improved from -inf to 0.53581, saving model to temp\model1_weights.h5
178/178 [==============================] - 178s 886ms/step - loss: 2.2147 - categorical_accuracy: 0.5358 - val_loss: 16159.0596 - val_categorical_accuracy: 0.3031 - lr: 0.0010
Epoch 2/100
178/178 [==============================] - ETA: 0s - loss: 0.9334 - categorical_accuracy: 0.6706
Epoch 2: categorical_accuracy improved from 0.53581 to 0.67065, saving model to temp\model1_weights.h5
178/178 [==============================] - 152s 855ms/step - loss: 0.9334 - categorical_accuracy: 0.6706 - val_loss: 113.4720 - val_categorical_accuracy: 0.4703 - lr: 0.0010
Epoch 3/100
178/178 [==============================] - ETA: 0s - loss: 1.1337 - categorical_accuracy: 0.7184
Epoch 3: categorical_accuracy improved from 0.67065 to 0.71840, saving model to temp\model1_weights.h5
178/178 [========

In [ ]:
if os.path.isfile('models/resnet152_2conv_train.h5') is False:
    model_ResNet.save('models/resnet152_2conv_train.h5')
    
resnet152_2conv_trained = load_model('models/resnet152_2conv_train.h5')

# Model 6 - InceptionV3


In [5]:
inception_conv = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in inception_conv.layers:
    layer.trainable = False    

inception_feedforword = Flatten()(inception_conv.output)
inception_feedforword = Dense(1024,activation = 'relu')(inception_feedforword)

prediction = Dense(len(folders), activation='softmax')(inception_feedforword)

model_inception = Model(inputs= inception_conv.input, outputs=prediction)

model_inception.compile(
  loss='categorical_crossentropy',
  optimizer='RMSprop',
  metrics=['categorical_accuracy']
)

model1_es = EarlyStopping(monitor = 'loss', min_delta = 1e-11, patience = 12, verbose = 1)
model1_rlr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 6, verbose = 1)
model1_mcp = ModelCheckpoint(filepath = r'temp\model1_weights.h5', monitor = 'categorical_accuracy',
                             save_best_only = True, verbose = 1)

model_inception_fit = model_inception.fit(
  train,
  validation_data=test,
  epochs=20,
  steps_per_epoch=5712//32,
  validation_steps=1311//32,
  callbacks=[model1_es, model1_rlr, model1_mcp])

87910968/87910968 [==============================] - 6s 0us/step
Epoch 1/20
178/178 [==============================] - ETA: 0s - loss: 15.0154 - categorical_accuracy: 0.7051
Epoch 1: categorical_accuracy improved from -inf to 0.70506, saving model to temp\model1_weights.h5
178/178 [==============================] - 110s 563ms/step - loss: 15.0154 - categorical_accuracy: 0.7051 - val_loss: 2.9590 - val_categorical_accuracy: 0.8047 - lr: 0.0010
Epoch 2/20
178/178 [==============================] - ETA: 0s - loss: 1.6660 - categorical_accuracy: 0.8097
Epoch 2: categorical_accuracy improved from 0.70506 to 0.80969, saving model to temp\model1_weights.h5
178/178 [==============================] - 72s 404ms/step - loss: 1.6660 - categorical_accuracy: 0.8097 - val_loss: 0.7484 - val_categorical_accuracy: 0.8234 - lr: 0.0010
Epoch 3/20
178/178 [==============================] - ETA: 0s - loss: 0.9642 - categorical_accuracy: 0.8065
Epoch 3: categorical_accuracy did not improve from 0.80969
178/